In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("db/Monarchs1998.db")

In [3]:
df = pd.read_csv("csv/1998 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1998", db, if_exists="replace")

os.getcwd()

C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1998_index,monarchs1998,3,"CREATE INDEX ""ix_monarchs1998_index""ON ""monarc..."
1,table,monarchs1998,monarchs1998,2,"CREATE TABLE ""monarchs1998"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1998")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,"Tag City, State",TEXT,0,None,0
4,4,Tagged,TEXT,0,None,0
5,5,Reported,TEXT,0,None,0
6,6,"Report City, State",TEXT,0,None,0
7,7,Reporter,TEXT,0,None,0
8,8,Miles,TEXT,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Reporter"], axis = 1, inplace = True)
df

,Tag No.,"Tag City, State",Tagged,Reported,"Report City, State",Miles
0,XA226,"Oshkosh, WI",09/25/98,04/06/99,"Rancho Viejo, TX",1814
1,WJ990,"Columbia, CT",08/29/98,03/01/99,"El Rosario, MX",2231
2,BG365,"Wallingford, CT",09/03/98,02/10/99,"El Rosario, MX",2199
3,WK478,"West Haven, CT",09/21/98,02/17/99,"El Rosario, MX",2186
4,WK499,"West Haven, CT",09/26/98,04/05/99,"Angangueo, MX",2186
...,...,...,...,...,...,...
454,DA348,NaN,NaN,03/01/99,"El Rosario, MX",NaN
455,DA382,NaN,NaN,03/01/99,"El Rosario, MX",NaN
456,NK076,NaN,NaN,02/10/99,"El Rosario, MX",NaN
457,SK300,NaN,NaN,02/19/99,"El Rosario, MX",NaN


### Rename columns to match other years

In [7]:
df.rename(columns={'Tagged': 'Tag Date', 'Reported': 'Report Date'}, inplace=True)

### Split & rename combined columns in Tagged category

In [8]:
df2 = df['Tag City, State'].str.split(', ', expand=True)
df2.columns = ['tag_city', 'Tag State']
df2.loc[df2['Tag State'] == 'NL (MX)', 'Tag State'] = 'Nuevo Leon'
df2.loc[df2['Tag State'] == 'COAH (MX)', 'Tag State'] = 'Coahuila'
df2.loc[df2['Tag State'] == 'ONT', 'Tag State'] = 'Ontario'
df2

,tag_city,Tag State
0,Oshkosh,WI
1,Columbia,CT
2,Wallingford,CT
3,West Haven,CT
4,West Haven,CT
...,...,...
454,NaN,NaN
455,NaN,NaN
456,NaN,NaN
457,NaN,NaN


### Split & Rename combined columns in Reported category

In [9]:
df3 = df['Report City, State'].str.split(', ', expand=True)
df3.columns = ['report_city', 'Report State']
df3.loc[df3['Report State'] == 'MX', 'Report State'] = 'Michoacán'
df3

,report_city,Report State
0,Rancho Viejo,TX
1,El Rosario,Michoacán
2,El Rosario,Michoacán
3,El Rosario,Michoacán
4,Angangueo,Michoacán
...,...,...
454,El Rosario,Michoacán
455,El Rosario,Michoacán
456,El Rosario,Michoacán
457,El Rosario,Michoacán


### Create new column Report Country & clean up

In [10]:
add_country = df['Report City, State'].str.extract("(MX)", expand=True)
add_country.columns = ['Report Country']
add_country.loc[add_country['Report Country'] == 'MX', 'Report Country'] = 'MEXICO'
add_country.loc[add_country['Report Country'].isnull(), 'Report Country'] = 'USA'
add_country

,Report Country
0,USA
1,MEXICO
2,MEXICO
3,MEXICO
4,MEXICO
...,...
454,MEXICO
455,MEXICO
456,MEXICO
457,MEXICO


### Create new table with unneeded columns dropped

In [11]:
df4 = pd.concat([df, df2, df3, add_country], axis=1)
df4.drop(['Tag City, State', 'Report City, State', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,XA226,WI,09/25/98,TX,USA,04/06/99,1814
1,WJ990,CT,08/29/98,Michoacán,MEXICO,03/01/99,2231
2,BG365,CT,09/03/98,Michoacán,MEXICO,02/10/99,2199
3,WK478,CT,09/21/98,Michoacán,MEXICO,02/17/99,2186
4,WK499,CT,09/26/98,Michoacán,MEXICO,04/05/99,2186
...,...,...,...,...,...,...,...
454,DA348,NaN,NaN,Michoacán,MEXICO,03/01/99,NaN
455,DA382,NaN,NaN,Michoacán,MEXICO,03/01/99,NaN
456,NK076,NaN,NaN,Michoacán,MEXICO,02/10/99,NaN
457,SK300,NaN,NaN,Michoacán,MEXICO,02/19/99,NaN


### Change Values in Report Date where day of month is missing
#### I chose 09/01/08 because season starts in August, and the average Tag date is in September

In [12]:
new_table.loc[new_table['Tag Date'] == "98 Season", 'Tag Date'] = '09/01/08'
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,XA226,WI,09/25/98,TX,USA,04/06/99,1814
1,WJ990,CT,08/29/98,Michoacán,MEXICO,03/01/99,2231
2,BG365,CT,09/03/98,Michoacán,MEXICO,02/10/99,2199
3,WK478,CT,09/21/98,Michoacán,MEXICO,02/17/99,2186
4,WK499,CT,09/26/98,Michoacán,MEXICO,04/05/99,2186
...,...,...,...,...,...,...,...
454,DA348,NaN,NaN,Michoacán,MEXICO,03/01/99,NaN
455,DA382,NaN,NaN,Michoacán,MEXICO,03/01/99,NaN
456,NK076,NaN,NaN,Michoacán,MEXICO,02/10/99,NaN
457,SK300,NaN,NaN,Michoacán,MEXICO,02/19/99,NaN


In [13]:
new_table['Season']='1998'

### Final check for unwanted spaces

In [14]:
# new_table.loc[new_table['Report State'].str.contains(r'/s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'/s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'/s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'/s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [15]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                     object
Season                    object
dtype: object

In [16]:
del df2, df3, df4, df5, add_country

### Export cleaned data to csv & pkl

In [17]:
new_table.to_pickle("pkl/Monarchs1998.pkl")
new_table.to_csv("csv/Monarchs1998.csv")